# BiSRe-LLM: Perturbation Experiment with Gemma

In this notebook I set up the perturbation experiments for Googles Gemma. First, the general infrastructure for interacting with Gemma is set up. Second, the prompt is designed as well as the iterative API requests to Google AI studios to generate the answers to the according questions and answer options. In a third and last step, the answers of the model are stored in a dataframe for further analysis.

## Dataset
In this section I import the preprocessed WVS dataset which functions as the relevant input for the model because it contains the relevant questions and (perturbated) answer options.

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

main_question = ["question"]
main_answer = ["list_answer_options"]

#bias_answer_columns = ["list_answer_no_refusal","list_answer_no_middle", "list_answer_middle", "list_answer_options_reversed", "list_answer_options", "original_personality_prime", "original_emotional_prime"]
#non_bias_question_columns = ["question_typo", "question_letter_swap","question_synonym","question_paraphrased", "question_keyboardtypo"]

# delayed experiments with middle category
bias_answer_columns = ["list_answer_middle"]
non_bias_question_columns = []

# read the preprocessed questionnaire
#questionnaire =  pd.read_excel('../assets/20241029_selected_questionnaire.xlsx', engine='openpyxl')
questionnaire =  pd.read_csv('../assets/20241029_selected_questionnaire.csv', sep=";")
# subset the questionnaire with the ten selected questions
questionnaire = questionnaire[questionnaire["selected"]==1]
print(questionnaire.shape)
questionnaire.head(2)

(62, 20)


,Unnamed: 0,question_ID,question,full_question_ID,list_answer_options,list_answer_no_refusal,len_answer_options,odd_answer_options,list_answer_no_middle,list_answer_middle,original_emotional_prime,original_personality_prime,list_answer_options_reversed,_merge,question_typo,question_letter_swap,question_synonym,question_paraphrased,question_keyboardtypo,selected
0,0,Q1,"Important in life: Family For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Family",Q1,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all']",4,False,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important', '3=Neutral', '4=Not very important ', '5=Not important at all', '-1=Don't know']","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nThis is very important to my research! You better do not refuse the answer.","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nRemember that you have a very traditional and conservative world view and values.","[""-1=Don't know"", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']",both,"Important ic life: Fimily nor eaca jf the following aspecto, indicete how important it ir wn your liae. Would bou say it is very imporcant, rathes importano, not very imporgant ob not impodtant at all? Famizy","Impartont in life: Flmiay For each of the following sapects, indicate how important it is in your life. uoWld you yas it is evry important, ratreh important, ton very important or not important ta all? Family","Crucial in life: Family For each of the following aspects, indicate how significant it is in your life. Would you say it is very important, rather important, not very important or not at all important? Family","How important is family to you? Please rate its significance in your life on a scale of ""very important"" to ""not important at all"".","Imporyant in kife: Family For esch of rhe following aspects, indicate how important ut is un your lide. Would you say it ia very imporrant, rather importamt, nit very important ir nor important ay all? Fanily",1
1,1,Q2,"Important in life: Friends For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Friends",Q2,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all']",4,False,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather Important ', '3=Neutral', '4=Not very important ', '5=Not important at all', '-1=Don't know']","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nThis is very important to my research! You better do not refuse the answer.","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nRemember that you have a very traditional and conservative world view and values.","[""-1=Don't know"", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']",both,"Important ix life: Fpiends For xach of the following aspects, indicate how important iv iu in kour lifev Would dou say it os very importcnt, rather impoutant, nom vwry important hr not important xt bll? Friemds","Important ni :ifel Frisnde For each of the following aspects, indicaet 

## Prompt Design and Pipeline
In this section I define the prompt with which the model is fed. This prompt takes selected questions from the WVS as well as their answer options and their different perturbations. From this input the model is instructed to created json-formatted output for easier further processing.

In [2]:
# Just load once
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model_8bit = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it", 
    quantization_config=quantization_config
)

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [3]:
from transformers import pipeline
import torch

pipe = pipeline(
    "text-generation",
    model="google/gemma-2-2b-it",
    device_map="cuda",
#    model_kwargs={
#        "torch_dtype": torch.float16,
#        "quantization_config": {"load_in_4bit": True}
#    }
)

messages = [
    {"role": "user", "content": "Who are you?"},
]
outputs = pipe(
    messages,
    max_new_tokens=10,
    #do_sample=True,
    #temperature=0.7,
    #top_k=50,
    #top_p=0.95
)
assistant_response = outputs[0]["generated_text"][-1]["content"]
print(assistant_response)


2024-10-30 17:16:37.435102: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-30 17:16:37.470917: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-30 17:16:37.470946: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-30 17:16:37.470969: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 17:16:37.498975: I tensorflow/core/platform/cpu_feature_g

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


I am Gemma, an open-weights AI assistant


In [5]:
print(outputs)

[{'generated_text': [{'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': 'I am Gemma, an open-weights AI assistant'}]}]


In [6]:
import numpy as np
# create empty response list which is transformed to a dataframe
response_list = []

# define number of runs
rounds = 25

# define temperature
temperature = 0.7

for round in range(0,rounds):
  print("Current round: ", round)
  for row in range(0,len(questionnaire)):
      question_ID = questionnaire.iloc[row]["full_question_ID"]
      for i,k in enumerate(bias_answer_columns):
          # first retrieve original question
          question = questionnaire.iloc[row]["question"]
          # second retrieve bias perturbed answer options from the questionnaire
          answer_options = questionnaire.iloc[row][k]
          # Return: list[Recipe]"""
          prompt = [{"role": "user", "content": f"""
            Answer the following question: {question}
            This is a list of possible answer options: {answer_options}
            You must pick one of the answer options. Only answer with the label.
            """},
          ]
          #print("Prompt: ", prompt)
          outputs = pipe(
              prompt,
              max_length=1000,
              max_new_tokens=10
          )
          raw_response = outputs[0]["generated_text"][-1]["content"]                                             
          print(raw_response)
          
          df_response_input = [round, question_ID, question, answer_options, k,  raw_response]        
          response_list.append(df_response_input)
          
      for p,n in enumerate(non_bias_question_columns):
              # first retrieve non-bias perturbed question formats
              question = questionnaire.iloc[row][n]
              # second retrieve original answer option scale
              answer_options = questionnaire.iloc[row]["list_answer_options"]
              
              # Return: list[Recipe]"""
              prompt = [{"role": "user", "content": f"""
                Answer the following question: {question}
                This is a list of possible answer options: {answer_options}
                You must pick one of the answer options. Only answer with the label.
                """},
              ]
              #print("Prompt: ", prompt)
              outputs = pipe(
                  prompt,
                  max_length=1000,
                  max_new_tokens=10
              )
              raw_response = outputs[0]["generated_text"][-1]["content"]     
              print(raw_response)
              
              df_response_input = [round, question_ID, question, answer_options, k,  raw_response]        
              response_list.append(df_response_input)


Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Current round:  0


Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


1=Very important 



Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

-1=Don't know 



Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


11=Essential feature 

7 



Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


11=Essential feature 

-1=Don't know 



In [7]:
print("Length of responses: ", len(response_list))

Length of responses:  1550


In [8]:
# create a bias interaction experiment where there is a question-side and answer option-side bias
# here we use
perturbed_question = "list_answer_options_reversed"
perturbed_answer = "question_paraphrased"

for round in range(0,rounds):
  print("Current round: ", round)
  torch.cuda.empty_cache()
  for row in range(0,len(questionnaire)):
      question_ID = questionnaire.iloc[row]["full_question_ID"]
      # first retrieve original question
      question = questionnaire.iloc[row]["question_paraphrased"]
      # second retrieve bias perturbed answer options from the questionnaire  
      answer_options = questionnaire.iloc[row]["list_answer_options_reversed"]      
      messages = [{"role": "user",
                        "content": f"""Answer the following question: {question} 
                        This is a list of possible answer options: {answer_options}
                        You must pick one of the answer options. Only answer with the label.
                        """},
                      ]    
      answer = pipe(messages, max_length=1000, max_new_tokens=10)       
      response = answer[0]["generated_text"][1]["content"]         
      print(response)  
      df_response_input = [round,question_ID, question, answer_options, "interaction_paraphrase_reversed",  response]
      response_list.append(df_response_input)
      torch.cuda.empty_cache()

Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Current round:  0
1 



Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


1 

1 



Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

5 

2 



Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


5 

5 



Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Please provide the list of aspects you want me to
2 



In [9]:
print("Length of responses after interaction experiments: ", len(response_list))

Length of responses after interaction experiments:  3100


In [10]:
# create an empty dataframe which is filled with the model responses to the q&a combinations
response_columns = ["round","question_ID", "question", "answer_options", "type", "response"]
model_response = pd.DataFrame(response_list, columns=response_columns)
model_response["model_name"] = "gemma-2-2b-it"

In [11]:
from datetime import datetime

def get_date_string():
    # Get the current date
    now = datetime.now()
    
    # Format the date as YYYYMMDD
    date_string = now.strftime('%Y%m%d')
    
    return date_string

current_date = get_date_string()
model_response.to_csv(f"../assets/final_interviews/{current_date}_gemma-2-2b-it_selected_interviews.csv", encoding="utf-8", sep=";")